# G4X 3D Reconstruction from Serial Sections

This notebook demonstrates how to reconstruct a 3D spatial transcriptomic and proteomic volume from serial sections acquired on the Singular Genomics G4X platform.

## Overview

The workflow includes:
1. Loading data from 9 serial sections
2. Aligning/registering sections across the z-stack
3. Building unified 3D coordinate space
4. Identifying 3D cellular niches
5. Tracking cells across sections
6. Interactive 3D visualization

## G4X Output File Structure

Each section directory contains:
```
section_XX/
├── rna/
│   ├── transcript_table.csv.gz      # Transcript locations (x, y, gene, cell_id)
│   └── raw_features.parquet         # Raw features pre-demux
├── single_cell_data/
│   ├── cell_metadata.csv.gz         # Cell centroids, areas, counts
│   ├── cell_by_transcript.csv.gz    # Cell x gene matrix
│   ├── cell_by_protein.csv.gz       # Cell x protein matrix (multiomics)
│   ├── clustering_umap.csv.gz       # Cluster assignments
│   └── feature_matrix.h5            # AnnData-compatible H5
├── segmentation/
│   └── segmentation_mask.npz        # nuclei, nuclei_exp masks
├── protein/
│   └── {protein}.jp2                # Protein images (multiomics)
├── h_and_e/
│   ├── h_and_e.jp2                  # fH&E image
│   ├── nuclear.jp2
│   └── cytoplasmic.jp2
└── metrics/
    └── transcript_core_metrics.csv
```

## Setup

In [ ]:
# Install required packages (if needed)
# !pip install pandas numpy scipy plotly scikit-learn pyarrow
# !pip install scanpy anndata  # Optional: for advanced analysis

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import our reconstruction module
from g4x_3d_reconstruction import G4X3DReconstructor
import g4x_3d_visualization as viz

# For inline plotly
import plotly.io as pio
pio.renderers.default = 'notebook'

## 1. Define Section Paths

Update these paths to point to your 9 serial section output directories.
**Important**: Order sections from bottom to top (z-order).

In [ ]:
# Define paths to your 9 serial sections (UPDATE THESE)
DATA_ROOT = Path("/path/to/your/g4x/data")

section_dirs = [
    DATA_ROOT / "section_01",
    DATA_ROOT / "section_02",
    DATA_ROOT / "section_03",
    DATA_ROOT / "section_04",
    DATA_ROOT / "section_05",
    DATA_ROOT / "section_06",
    DATA_ROOT / "section_07",
    DATA_ROOT / "section_08",
    DATA_ROOT / "section_09",
]

# Verify paths exist
for p in section_dirs:
    exists = p.exists()
    print(f"{p.name}: {'✓' if exists else '✗ NOT FOUND'}")

## 2. Initialize Reconstructor

Configure the physical parameters:
- `section_thickness`: FFPE sections are typically 4-5 µm
- `pixel_size`: G4X pixel size (check your run metadata)

In [ ]:
reconstructor = G4X3DReconstructor(
    section_dirs=[str(d) for d in section_dirs],
    section_thickness=5.0,   # µm per section
    pixel_size=0.5           # µm per pixel
)

print(f"Initialized reconstructor with {reconstructor.n_sections} sections")
print(f"Expected volume depth: {reconstructor.n_sections * 5.0} µm")

## 3. Load All Section Data

In [ ]:
# Load data from all sections
reconstructor.load_sections(verbose=True)

## 4. Quality Check: Visualize Individual Sections

In [ ]:
# Plot individual sections side by side
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=[f"Section {i}" for i in range(9)]
)

for i, section in enumerate(reconstructor.sections):
    if section.transcripts is not None:
        # Subsample for visualization
        tx = section.transcripts.sample(n=min(5000, len(section.transcripts)))
        
        row = i // 3 + 1
        col = i % 3 + 1
        
        fig.add_trace(
            go.Scatter(
                x=tx['x'],
                y=tx['y'],
                mode='markers',
                marker=dict(size=1, opacity=0.5),
                name=f"S{i}"
            ),
            row=row, col=col
        )

fig.update_layout(
    height=800,
    title="Individual Sections (Before Alignment)",
    showlegend=False
)
fig.show()

## 5. Align Sections

Register sections to a common coordinate system. Options:
- `'centroid'`: Fast, aligns by tissue centroid
- `'icp'`: Iterative Closest Point on cell centroids (more accurate)

In [ ]:
# Align sections using centroid method first (fast)
reconstructor.align_sections(
    method='centroid',      # 'centroid' or 'icp'
    reference_index=4,      # Use middle section as reference
    verbose=True
)

In [ ]:
# Visualize alignment quality
fig = viz.plot_alignment_quality(reconstructor.sections)
fig.show()

## 6. Build 3D Volume

In [ ]:
# Construct unified 3D coordinate space
reconstructor.build_3d_volume(verbose=True)

In [ ]:
# Quick look at the 3D data
print("\n3D Transcripts:")
print(reconstructor.transcripts_3d.head())
print(f"\nShape: {reconstructor.transcripts_3d.shape}")

print("\n3D Cells:")
print(reconstructor.cells_3d.head())
print(f"\nShape: {reconstructor.cells_3d.shape}")

## 7. 3D Visualization: Transcripts

In [ ]:
# 3D scatter plot of transcripts (colored by section)
fig = viz.plot_transcripts_3d(
    reconstructor.transcripts_3d,
    color_by='section_index',
    max_points=50000,
    point_size=2,
    title="3D Transcript Distribution (colored by section)"
)
fig.show()

In [ ]:
# Visualize specific genes in 3D
# Update with genes from your panel
genes_of_interest = ['CD3E', 'CD8A', 'CD4', 'FOXP3']  # Example immune markers

fig = viz.plot_transcripts_3d(
    reconstructor.transcripts_3d,
    color_by='gene_name',
    genes=genes_of_interest,
    max_points=30000,
    title="T Cell Markers in 3D"
)
fig.show()

## 8. 3D Visualization: Cells

In [ ]:
# 3D scatter plot of cells (colored by cluster)
fig = viz.plot_cells_3d(
    reconstructor.cells_3d,
    color_by='section_index',  # Will auto-detect cluster columns
    size_by='total_counts',
    title="3D Cell Distribution"
)
fig.show()

## 9. Identify 3D Niches

Cluster cells based on their 3D neighborhood composition to identify tissue microenvironments.

In [ ]:
# Find 3D niches
cells_with_niches = reconstructor.find_3d_niches(
    n_neighbors=30,     # Neighborhood size
    resolution=0.5,     # Clustering resolution
    verbose=True
)

In [ ]:
# Visualize 3D niches
fig = viz.plot_niche_distribution(
    reconstructor.cells_3d,
    niche_col='niche_3d'
)
fig.show()

In [ ]:
# Niche composition analysis
print("\nNiche Statistics:")
print(reconstructor.cells_3d['niche_3d'].value_counts().sort_index())

# If cluster annotations are available
cluster_cols = [c for c in reconstructor.cells_3d.columns if 'leiden' in c.lower()]
if cluster_cols:
    print("\n\nNiche x Cell Type Cross-tabulation:")
    print(pd.crosstab(
        reconstructor.cells_3d['niche_3d'],
        reconstructor.cells_3d[cluster_cols[0]],
        normalize='index'
    ).round(2))

## 10. Track Cells Across Sections

In [ ]:
# Track cells between adjacent sections
cell_tracks = reconstructor.track_cells_across_sections(
    max_distance=50.0,  # Maximum distance for matching (pixels)
    verbose=True
)

print(f"\nTotal cell matches: {len(cell_tracks)}")
print(cell_tracks.head(10))

## 11. Gene Expression in 3D

In [ ]:
# Visualize a specific gene's expression in 3D
# Update with a gene from your panel
gene = 'CD8A'  # Example

try:
    fig = viz.plot_gene_expression_3d(
        reconstructor.transcripts_3d,
        reconstructor.cells_3d,
        gene=gene,
        aggregation='count'
    )
    fig.show()
except ValueError as e:
    print(f"Gene not found: {e}")
    print("\nAvailable genes (first 50):")
    print(reconstructor.transcripts_3d['gene_name'].unique()[:50])

## 12. Section Comparison

In [ ]:
# Compare sections side by side
fig = viz.plot_section_comparison(
    reconstructor.cells_3d,
    feature='total_counts',
    n_cols=3
)
fig.show()

## 13. Export Results

In [ ]:
# Define output directory
OUTPUT_DIR = Path("./g4x_3d_output")
OUTPUT_DIR.mkdir(exist_ok=True)

# Save processed data
saved_files = reconstructor.save_results(
    str(OUTPUT_DIR),
    formats=['parquet', 'csv'],
    verbose=True
)

In [ ]:
# Export for napari visualization
napari_exports = reconstructor.export_for_napari(
    str(OUTPUT_DIR / "napari"),
    verbose=True
)

In [ ]:
# Export for Vitessce web visualization
vitessce_exports = reconstructor.export_for_vitessce(
    str(OUTPUT_DIR / "vitessce"),
    verbose=True
)

In [ ]:
# Create interactive HTML dashboard
dashboard_path = viz.create_dashboard(
    reconstructor,
    output_path=str(OUTPUT_DIR / "g4x_3d_dashboard.html")
)
print(f"\nDashboard saved to: {dashboard_path}")

## 14. Advanced: Load in scanpy for Further Analysis

In [ ]:
try:
    import scanpy as sc
    import anndata as ad
    
    # Create AnnData from 3D cells
    # Would need expression matrix from sections
    print("scanpy available for further analysis")
    
    # Load feature matrices and concatenate
    adatas = []
    for section in reconstructor.sections:
        h5_path = Path(section.section_id) / "single_cell_data" / "feature_matrix.h5"
        if h5_path.exists():
            adata = sc.read_h5ad(h5_path)
            adata.obs['section'] = section.section_id
            adatas.append(adata)
    
    if adatas:
        adata_combined = ad.concat(adatas)
        print(f"Combined AnnData: {adata_combined.shape}")
        
except ImportError:
    print("scanpy not installed. Install with: pip install scanpy")

## Summary

This notebook demonstrated the complete workflow for 3D reconstruction from G4X serial sections:

1. ✓ Loaded 9 serial sections
2. ✓ Aligned sections to common coordinate system
3. ✓ Built unified 3D transcript and cell coordinates
4. ✓ Identified 3D cellular niches
5. ✓ Tracked cells across sections
6. ✓ Created interactive 3D visualizations
7. ✓ Exported for external tools (napari, Vitessce)

### Next Steps

- **Fine-tune alignment**: Use ICP method or image-based registration for better accuracy
- **Niche characterization**: Analyze gene/protein composition per niche
- **3D spatial statistics**: Compute co-localization, gradients across z
- **Integration**: Combine with H&E images for morphological context